# Setup

In [1]:
import jnius_config

In [2]:
jnius_config.set_classpath("../../java/target/demo-python-java-api-1.0-SNAPSHOT-jar-with-dependencies.jar")
# See https://github.com/kivy/pyjnius/issues/283
jnius_config.add_options('-Xss1280k')
jnius_config.add_options('-XX:-UseParallelGC')

In [3]:
import jnius

import cProfile
import time
import pandas as pd
import random

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import sys 
sys.path.append('..')
import api.demojnius

In [5]:
Simulation = jnius.autoclass('ch.dubernet.demopythonapi.simulation.Simulation')
BenchmarkEventHandler = jnius.autoclass('ch.dubernet.demopythonapi.simulation.events.BenchmarkEventHandler')

JumpEventHandler = 'ch/dubernet/demopythonapi/simulation/events/JumpEventHandler'
SingEventHandler = 'ch/dubernet/demopythonapi/simulation/events/SingEventHandler'
SpeakEventHandler = 'ch/dubernet/demopythonapi/simulation/events/SpeakEventHandler'

In [6]:
N_AGENTS = 100
N_TIME_STEPS = 100
N_TRIES = 100

In [7]:
class Stopwatch:
    def __init__(self):
        self._records = []
    
    def start(self):
        self._start_time = time.time()
        
    def end(self, **attributes):
        t = time.time()
        self._records.append({**attributes, "time": t - self._start_time})
    
    def to_frame(self):
        return pd.DataFrame.from_records(self._records)

stopwatch = Stopwatch()

# Run using Java only

In [ ]:
simulation = Simulation(N_AGENTS, N_TIME_STEPS)
simulation.getEvents().addEventHandler(BenchmarkEventHandler())

for i in range(N_TRIES):
    stopwatch.start()
    simulation.run()
    stopwatch.end(setting="pure java", n_agents=N_AGENTS, n_time_steps=N_TIME_STEPS)
    
print(stopwatch.to_frame().query("setting == \"pure java\"").time.mean())

pr = cProfile.Profile()
pr.enable()
simulation.run()
pr.disable()
pr.print_stats()

Pretty much the same as with JPype

# Naive Python Implementation

In [ ]:
simulation = Simulation(N_AGENTS, N_TIME_STEPS)

class PythonBenchmarkHandler(jnius.PythonJavaClass):
    __javainterfaces__ = [JumpEventHandler, SingEventHandler, SpeakEventHandler]

    @jnius.java_method('()V')
    def notifyStart(self):
        pass
    
    @jnius.java_method('()V')
    def notifyEnd(self):
        pass
    
    @jnius.java_method('(Lch/dubernet/demopythonapi/simulation/events/JumpEvent;)V', name='handleEvent')
    def handleJumpEvent(self, event):
            event.getAgentId()
            event.getHeight_m()
            event.getTime()

            
    @jnius.java_method('(Lch/dubernet/demopythonapi/simulation/events/SingEvent;)V', name='handleEvent')
    def handleSingEvent(self, event):
            event.getAgentId()
            event.getSong()
            event.getTime()

            
    @jnius.java_method('(Lch/dubernet/demopythonapi/simulation/events/SpeakEvent;)V', name='handleEvent')
    def handleSpeakEvent(self, event):
            event.getAgentId()
            event.getMessage()
            event.getTime()

handler = PythonBenchmarkHandler()
simulation.getEvents().addEventHandler(handler)

for i in range(N_TRIES):
    stopwatch.start()
    simulation.run()
    stopwatch.end(setting="pure python", n_agents=N_AGENTS, n_time_steps=N_TIME_STEPS)
    
print(stopwatch.to_frame().query("setting == \"pure python\"").time.mean())
            
pr = cProfile.Profile()
pr.enable()
simulation.run()
pr.disable()
pr.print_stats()

That is roughly twice as bad as JPype... Let's see what the figure becomes once we reduce communication to the minimum.

# With Protocol Buffers

In [9]:
simulation = Simulation(N_AGENTS, N_TIME_STEPS)

class PythonPbBenchmarkHandler:    
    def handleJumpEvent(self, event):
            event.agentId
            event.height_m
            event.time
    
    def handleSingEvent(self, event):
            event.agentId
            event.song
            event.time
            
    def handleSpeakEvent(self, event):
            event.agentId
            event.text
            event.time

handler = PythonPbBenchmarkHandler()
adapter = api.demojnius.create_event_handler(handler)
simulation.getEvents().addEventHandler(adapter)

for i in range(N_TRIES):
    stopwatch.start()
    simulation.run()
    stopwatch.end(setting="pb python", n_agents=N_AGENTS, n_time_steps=N_TIME_STEPS)
    
print(stopwatch.to_frame().query("setting == \"pb python\"").time.mean())
            
pr = cProfile.Profile()
pr.enable()
simulation.run()
pr.disable()
pr.print_stats()

1.0965033042430878
         200049 function calls in 1.213 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    20004    0.013    0.000    0.019    0.000 <frozen importlib._bootstrap>:416(parent)
    20004    0.007    0.000    0.026    0.000 <frozen importlib._bootstrap>:997(_handle_fromlist)
     3364    0.005    0.000    0.005    0.000 <ipython-input-9-a543bf37debf>:14(handleSpeakEvent)
        1    0.978    0.978    1.213    1.213 <ipython-input-9-a543bf37debf>:32(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-9-a543bf37debf>:33(<module>)
     3367    0.003    0.000    0.003    0.000 <ipython-input-9-a543bf37debf>:4(handleJumpEvent)
     3269    0.004    0.000    0.004    0.000 <ipython-input-9-a543bf37debf>:9(handleSingEvent)
        2    0.000    0.000    0.000    0.000 codeop.py:132(__call__)
        1    0.000    0.000    0.000    0.000 demojnius.py:17(notifyStart)
        1    0.000    0.000  